In [2]:
#part 1 - version 4:  To get all Dec data _ keep original codes
import requests
import re
import pandas as pd
import pprint                  # pretty print
import datetime
#from datetime import datetime   # to do time calculation  #convert to datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')    
from datetime import timedelta

date_base = datetime.date(2020, 12, 1)
num_days = 31
df_list_all_date = []

for x in range(num_days):
    Date = date_base + datetime.timedelta(days=x)
    print(str(Date), ' with type of :' , type(Date))
    url = "https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=" + str(Date) + "&lang=en&cargo=false&arrival=false"
    print(url)
    querystring = {"Date":str(Date)}     # Date: String: “YYYY-MM-DD” ; Arrival: (true=arrival, false=departure); Cargo (true=cargo, false=passenger)

    response = requests.request("GET", url, params=querystring)
    #pprint.pprint(response.text)
    df = pd.read_json(response.text)
    df_backup = df
    df1 = pd.DataFrame( df['list'][0])                  ### To turn the data into DataFrame & remove cancelled flight
    df1['dept_date']=Date
    df_temp = df1[df1['statusCode']=='DA']                #Remove::  CX = Cancelled , DU = Delayed to another day
    if x == 0:
        df_all_date = df_temp
    else:
        df_all_date= pd.concat([df_all_date , df_temp ])
    df_list_all_date.append(df_temp)
    ###print(df_temp)

    #df_temp.to_csv('air_Raw_' + str(Date) + '.csv' , index = None)  #---->to csv file 
#for y in range(1, df_all_date):
#df_all_date = pd.concat(df_list_all_date)
print(df_all_date)
df_all_date.to_csv('air' + '_Raw_202012v4' + '.csv' , index = None)

2020-12-01  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-01&lang=en&cargo=false&arrival=false
2020-12-02  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-02&lang=en&cargo=false&arrival=false
2020-12-03  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-03&lang=en&cargo=false&arrival=false
2020-12-04  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-04&lang=en&cargo=false&arrival=false
2020-12-05  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-05&lang=en&cargo=false&arrival=false
2020-12-06  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-06&lang=en&cargo=false&arrival=fals

In [8]:
df_temp = pd.read_csv('air' + '_Raw_202012v4' + '.csv')
df_temp

,destination,terminal,aisle,gate,time,flight,statusCode,status,dept_date
0,['FRA'],T1,B,64.0,00:35,"[{'no': 'CX 289', 'airline': 'CPA'}, {'no': 'L...",DA,Dep 01:09,2020-12-01
1,['AMS'],T1,B,47.0,00:50,"[{'no': 'CX 271', 'airline': 'CPA'}]",DA,Dep 01:08,2020-12-01
2,['LHR'],T1,NaN,5.0,01:10,"[{'no': 'BA 032F', 'airline': 'BAW'}]",DA,Dep 01:34,2020-12-01
3,['LHR'],T1,NaN,45.0,01:30,"[{'no': 'BA 028F', 'airline': 'BAW'}]",DA,Dep 01:22,2020-12-01
4,['TLV'],T1,B,26.0,01:30,"[{'no': 'CX 675', 'airline': 'CPA'}]",DA,Dep 01:21,2020-12-01
...,...,...,...,...,...,...,...,...,...
3003,['HGH'],T1,K,NaN,21:50,"[{'no': 'HX 128', 'airline': 'CRK'}, {'no': 'H...",DA,Dep 21:47,2020-12-31
3004,['TPE'],T1,NaN,8.0,22:40,"[{'no': 'CI 922', 'airline': 'CAL'}]",DA,Dep 22:17,2020-12-31
3005,['KUL'],T1,K,27.0,23:00,"[{'no': 'HX 2747', 'airline': 'CRK'}]",DA,Dep 22:45,2020-12-31
3006,['IST'],T1,NaN,9.0,23:05,"[{'no': 'TK 6081', 'airline': 'THY'}]",DA,Dep 22:49,2020-12-31


In [64]:
#part 2 - version 2:    To work on data based on Dec csv file as dataframe called df_temp
#doc:   https://www.hongkongairport.com/iwov-resources/misc/opendata/Flight_Information_DataSpec_en.pdf
#site:  https://data.gov.hk/en-data/dataset/aahk-team1-flight-info
import requests
import re
import pandas as pd
import numpy as np
import pprint                   
from datetime import datetime      
from datetime import timedelta  #add one day 
import json 

df_temp = df_all_date       #continue to work on the data above
#df_temp = pd.read_csv('air' + '_Raw_202012v4' + '.csv')

#---------------------------------------Data processing---------------------------------------
# destination column
df_temp['destination'] = df_temp['destination'].astype('str')  
df_temp['destination'] = df_temp['destination'].str.extract(r'([A-Z]+)',expand = True)

# flight number column
df_temp['flight number'] = df_temp['flight'].apply( lambda x: x[0]['no'])

# airline column
df_temp['airline'] = df_temp['flight'].apply( lambda x: x[0]['no'][0:2])        

#actual_dept_time column
df_temp['est_dept_time'] = df_temp['time'].apply( lambda x: datetime.strptime(x, '%H:%M'))
df_temp['day'] = df_temp['status'].apply( lambda x: timedelta(days=1) if '(' in x else timedelta(days=0) )
df_temp['actual_dept_time'] = df_temp['status'].str.extract(r'Dep (..:..)')
df_temp['actual_dept_time'] = df_temp['actual_dept_time'].apply( lambda x: datetime.strptime(x, '%H:%M'))  #convert to datetime format

#delay column
df_temp['delay_time'] = df_temp['actual_dept_time'] - df_temp['est_dept_time']  + df_temp['day']            
df_temp['delay_time'] = df_temp['delay_time'].apply( lambda x: x if x > timedelta(days=0) else timedelta(days=0))
#---------------------------------------Data processing (Done) -------------------------------

#---------------------------------------File export-------------------------------------------
df_temp.to_csv('air' + '_Finalizing_202012v4' + '.csv' , index = None)


#------------------------------Data type testing-------------------------------------
print('type of a column: ', type(df_temp['delay_time'][0:1]) , ' | value of a column: ', df_temp['delay_time'][0:1] )
print('type of mean of a column: ', type(df_temp['delay_time'][0:1].mean()) , ' | value of mean of a column: ', df_temp['delay_time'][0:1].mean() )
#datetime_object = datetime.strptime(df_temp['time'][0], '%H:%M')
#print(datetime_object)
#print('type of a column: ', type(df_temp['flight'][0]) , ' | value of a column: ', df_temp['flight'][0] )
#------------------------------Data type testing-------------------------------------


print( ' This is group by type:  ' , type(  df_temp.groupby('destination').delay_time ) )
print( '\n\n---Max delay time of each destination is: ---\n' , df_temp.groupby('destination')['delay_time'].max() )
#print( '\n\n---Average delay time of each destination is: ---\n' , list( df_temp.groupby('destination')['delay_time']) )

# agg = lambda x: (x['Timestamp'].astype('i8') * (x['Value'].astype('f8') / x['Value'].sum())).sum()
# new_table = table.groupby(level='ID').apply(agg).astype('i8').astype('datetime64[ns]')

df_temp.head() 



type of a column:  <class 'pandas.core.series.Series'>  | value of a column:  0   00:34:00
Name: delay_time, dtype: timedelta64[ns]
type of mean of a column:  <class 'pandas._libs.tslibs.timedeltas.Timedelta'>  | value of mean of a column:  0 days 00:34:00
 This is group by type:   <class 'pandas.core.groupby.generic.SeriesGroupBy'>


---Max delay time of each destination is: ---
 destination
ADD   0 days 01:06:00
AKL   0 days 00:19:00
AMM   1 days 23:55:00
AMS   0 days 00:43:00
BKK   0 days 11:04:00
            ...      
VKO   0 days 16:24:00
XMN   0 days 01:05:00
YVR   0 days 00:21:00
YYZ   0 days 01:07:00
ZRH   0 days 00:41:00
Name: delay_time, Length: 68, dtype: timedelta64[ns]


,destination,terminal,aisle,gate,time,flight,statusCode,status,dept_date,flight number,airline,est_dept_time,day,actual_dept_time,delay_time
0,FRA,T1,B,64,00:35,"[{'no': 'CX 289', 'airline': 'CPA'}, {'no': 'L...",DA,Dep 01:09,2020-12-01,CX 289,CX,1900-01-01 00:35:00,0 days,1900-01-01 01:09:00,00:34:00
1,AMS,T1,B,47,00:50,"[{'no': 'CX 271', 'airline': 'CPA'}]",DA,Dep 01:08,2020-12-01,CX 271,CX,1900-01-01 00:50:00,0 days,1900-01-01 01:08:00,00:18:00
2,LHR,T1,,5,01:10,"[{'no': 'BA 032F', 'airline': 'BAW'}]",DA,Dep 01:34,2020-12-01,BA 032F,BA,1900-01-01 01:10:00,0 days,1900-01-01 01:34:00,00:24:00
3,LHR,T1,,45,01:30,"[{'no': 'BA 028F', 'airline': 'BAW'}]",DA,Dep 01:22,2020-12-01,BA 028F,BA,1900-01-01 01:30:00,0 days,1900-01-01 01:22:00,00:00:00
4,TLV,T1,B,26,01:30,"[{'no': 'CX 675', 'airline': 'CPA'}]",DA,Dep 01:21,2020-12-01,CX 675,CX,1900-01-01 01:30:00,0 days,1900-01-01 01:21:00,00:00:00


In [65]:
table = soup.find("table", attrs={"class":"datatable"})
#print(table)
# The first tr contains the field names.
headings = [td.get_text() for td in table.find("tr").find_all("td")]
datasets = []

import time, logging, threading

for x in range(1,22,1):
    link='?airline.page=' + str(x) + '&airline.search='
    URL = "https://www.iata.org/en/publications/directories/code-search/" + link
    print(URL)
    html = requests.get(URL ,timeout=(5,5)) #, timeout=5
    #time.sleep(5)
    #threading.Event.wait(5)
    print(html)
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find("table", attrs={"class":"datatable"})
    for row in table.find_all("tr")[1:]:
        dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
        #print("dataset is: " ,list(dataset))
        datasets.append(list(dataset))
import pprint
pprint.pprint(datasets)

NameError: name 'table' is not defined